In [11]:
import pandas as pd
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [20]:
df = pd.read_csv("joined-weather-solar.csv")
df['Date'] = pd.to_datetime(df['Date'])
df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month
df['day'] = df['Date'].dt.month
df = df.drop(columns=['Date','Station pressure', 'Unnamed: 0', 'Altimeter'])
df = df.rename(columns={'Temperature':'temp',
                        'Dew point':'dew',
                        'Wind speed':'windspeed',
                        'Cloud coverage': 'cloudcover',
                        'Visibility': 'visibility',
                        'Solar energy': 'solarenergy',
                        'Relative humidity':'humidity'})
X = df.drop(columns=['Site Performance Estimate'], axis=1)
y = df['Site Performance Estimate']

# Splitting the dfset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizing the df
scaler = MinMaxScaler(feature_range=(0, 1))
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [21]:
# Read data from API
vs_test = pd.read_csv("sample_visual_crossing_df.csv")

# Preprocess the data
vs_test['datetime'] = pd.to_datetime(vs_test['datetime'])
vs_test['year'] = vs_test['datetime'].dt.year
vs_test['month'] = vs_test['datetime'].dt.month
vs_test['day'] = vs_test['datetime'].dt.day
vs_test = vs_test.drop(columns=['datetime', 'name'])

# Reorder column and MinMaxScaler
vs_test = vs_test[['cloudcover', 'visibility', 'temp', 'dew', 'humidity', 'windspeed',
       'solarenergy', 'year', 'month', 'day']]
vs_test = scaler.transform(vs_test)

In [22]:
# Predict using AdaBoosting
final_ada = AdaBoostRegressor(base_estimator=None, 
                              learning_rate=0.5, 
                              n_estimators=150,
                              random_state=42)
result = final_ada.fit(X_train, y_train)

# Final prediction
vs_pred = result.predict(vs_test)

In [29]:
vs_output = pd.read_csv("Data/sample_visual_crossing_df.csv")
vs_output['predicted_solar'] = vs_pred
vs_output.to_csv("sample_output.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'Data/sample_visual_crossing_df.csv'

In [27]:
vs_test

array([[ 4.81000000e+01,  1.68926554e+00,  6.17598098e-01,
         5.13437719e-01,  5.13707572e-01,  1.04495798e+00,
        -1.44308584e-02,  7.00000000e+00,  9.09090909e-01,
         1.54545455e+00],
       [ 9.00000000e+00,  1.70056497e+00,  5.08204518e-01,
         3.98924982e-01,  4.62793734e-01,  8.18067227e-01,
        -1.43527039e-02,  7.00000000e+00,  9.09090909e-01,
         1.63636364e+00],
       [ 2.67000000e+01,  1.71186441e+00,  4.46373365e-01,
         3.10119187e-01,  4.00130548e-01,  7.29831933e-01,
        -1.43678306e-02,  7.00000000e+00,  9.09090909e-01,
         1.72727273e+00],
       [ 7.36000000e+01,  1.65536723e+00,  4.82045184e-01,
         4.17620939e-01,  5.63315927e-01,  1.52815126e+00,
        -1.45190974e-02,  7.00000000e+00,  9.09090909e-01,
         1.81818182e+00],
       [ 6.91000000e+01,  2.07344633e+00,  6.00951249e-01,
         6.11591493e-01,  8.02219321e-01,  1.34747899e+00,
        -1.45569142e-02,  7.00000000e+00,  9.09090909e-01,
         1.